In [3]:
suppressPackageStartupMessages({
    # Tidyverse
    library('dplyr')
    library('readr')
    library('purrr')
    library('glue')
    library('stringr')
    library('tibble')

    # Bioinformatics - Helpers
    library('biomaRt')  # ID-mapping
    library('tximport')  # For smooth kallisto imports

    # Bioinformatics - Analyses tools
    library('estimate')  # http://bioinformatics.mdanderson.org/estimate/
    library('xCell')  # https://github.com/dviraran/xCell
    library('MCPcounter')
    # ... and we have CIBERSORT as a Java/R hybrid application
})

# Show traceback on runtime error!
options(error=function() { traceback(2); q() })

# Other helpers
head3 <- function(x, ...) { head(x, n=3, ...) }

## simple wrapper for running shell commands
shell <- function(cmdargs, command="bash") {
    # We need the full path to the main program
    command_path <- Sys.which(command)
    system(glue("{command_path} {cmdargs}"))
}

In [34]:
get_expression_data <- function(tmpdir) {
    expression_name <- glue("{tmpdir}/gene-expression-matrix.csv")
    if (file.exists(expression_name)) {
        return(read_csv(expression_name) %>%
                    data.frame() %>%
                    tibble::column_to_rownames("Gene_symbol"))
    }

    mart <- useMart(biomart="ENSEMBL_MART_ENSEMBL", dataset="hsapiens_gene_ensembl", host="ensembl.org")
    t2g <- getBM(attributes=c("ensembl_transcript_id", "external_gene_name"), mart=mart)
    t2g <- dplyr::rename(t2g, target_id=ensembl_transcript_id, ext_gene=external_gene_name)
    head(t2g)

    kallisto_files <- dir('data', recursive=TRUE, full.names=TRUE, pattern="*.tsv")

    # Import and reduce
    txi <- tximport(kallisto_files, type="kallisto", tx2gene=t2g)
    expression_data <- txi$counts

    # RCC and Kallisto names are redundant and can be omitted
    # Makes columns patient identifiers
    colnames(expression_data) <- kallisto_files %>% 
                                    str_replace("-kallisto/abundance.tsv", "") %>%
                                    str_replace("data/", "") %>%
                                    str_replace_all("-|_", ".")
    head3(expression_data)

    expression_data %>%
        as.data.frame %>%
        tibble::rownames_to_column("Gene_symbol") %>%
        write_csv(path=expression_name)
}

In [35]:
run_cibersort <- function(expression_data, tmpdir) {
    cibersort_home_dir <- glue("{tmpdir}/cibersort")

    lm22_path <- glue("{cibersort_home_dir}/LM22.txt")
    lm22_df <- read_tsv(lm22_path)
    head3(lm22_df)

    lm22_genes <- lm22_df %>% dplyr::select(`Gene symbol`) %>% flatten_chr
    lm22_rel_genes <- intersect(rownames(expression_data), lm22_genes)
    print(
      paste(
        length(lm22_rel_genes), "out of", length(lm22_genes),
        "LM22 genes are present in the expression data"
      )
    )

    # Filter the matrix down to LM22 genes
    expression_data_lm22 <- expression_data[lm22_rel_genes, ]
    head(expression_data_lm22)
    dim(expression_data_lm22)

    # Filter out genes where we have almost no data (i.e. top 10%), and write out as
    # an input file for CIBERSORT.
    cibersort_input <- glue("{tmpdir}/cibersort-input.tsv")
    low_expression_threshold <- 1
    test <- expression_data_lm22 %>%
        data.frame() %>%
        tibble::rownames_to_column("Gene_symbols") %>%
        mutate(low_cnt=rowSums(.[-1] < low_expression_threshold)) %>%
        filter(low_cnt < quantile(low_cnt, .90)) %>%  # FIXME: Changed from `<` to `<=`...didn't work when low_cnt was 0 for most/all.
        dplyr::select(-low_cnt) %>%
        write_tsv(cibersort_input) %>%
        dim() # +1 column since we added the indices back to table for convenience

    # Run CIIBERSORT
    library(Rserve)
    Rserve(args="--no-save")
    shell(
        command="java",
        glue(
         "-Xmx10g -Xms3g \\
          -jar {cibersort_home_dir}/CIBERSORT.jar \\
          -M {cibersort_input} -B {lm22_path} \\
          > {tmpdir}/cibersort-output.tsv
          2> {tmpdir}/cibersort.err.log"
        )  # by default results are printed to stdout
    )
    shell(command="pkill", "-9 Rserve")  # no need to be graceful here

    # Read in results
    cibersort <- read_delim(glue("{tmpdir}/cibersort-output.tsv"), delim="\t", comment=">", trim_ws=TRUE)
    head(cibersort)

    # The CIBERSORT output has an extra tab at the end of the headers row, so it adds
    # in this extra column `X27` with no data. The output also contains a column called
    # `Column` that just numbers each row. Remove both of these here.
    cibersort.df <- cibersort %>%
                        data.frame %>%
                        dplyr::select(-`X27`, -`Column`)
    sampleNames <- colnames(expression_data_lm22)
    row.names(cibersort.df) <- sampleNames
    
    cibersort.df %>%
        tibble::rownames_to_column("Sample Name") %>%
        write_tsv(glue("{tmpdir}/cibersort-output.tsv"))

    cibersort.df
}

In [24]:
run_estimate <- function(expression_data, tmpdir) {
    # Convert our expression matrix into GCT format and save it to be 
    # used by ESTIMATE
    estimate_input <- glue("{tmpdir}/estimate-input.gct")

    outputGCT(expression_data, estimate_input)

    # Run ESTIMATE and save the results into another GCT file
    estimate_output <- glue('{tmpdir}/estimate-output.gct')
    estimateScore(estimate_input, output.ds=estimate_output, platform="illumina")

    # Load them back and return
    estimate_df <- read_tsv(estimate_output, comment="#", skip=1) %>%
                        dplyr::select(-`NAME`) %>%
                        as.data.frame
    estimate_df %>%
        column_to_rownames("Description")
}

In [16]:
run_xcell <- function(expression_data, tmpdir) {
    xcell_res <- xCellAnalysis(expression_data,
                               save.raw=TRUE,  # This produces a second output file with untransformed scores.
                               file.name=glue('{tmpdir}/xcell-output'))
    
    xcell_output <- glue('{tmpdir}/xcell-output.tsv')
    xcell_res %>%
        as.data.frame %>%
        rownames_to_column("CellType") %>%
        write_tsv(xcell_output)
    xcell_res
}

In [37]:
run_mcpcounter <- function(expression_data, tmpdir) {
    mcp_res <- MCPcounter.estimate(expression_data, featuresType="HUGO_symbols")
    mcp_output <- glue("{tmpdir}/mcpcounter-output.tsv")
    mcp_res %>%
        as.data.frame %>%
        rownames_to_column("CellType") %>%
        write_tsv(mcp_output)
    mcp_res
}

In [38]:
tmpdir <- "build"
expression_data <- get_expression_data(tmpdir)
head3(expression_data)
# run_cibersort(expression_data, tmpdir)
# run_estimate(expression_data %>% data.frame() %>% tibble::column_to_rownames('Gene_symbol'), tmpdir)
# run_estimate(expression_data, tmpdir)
# run_xcell(expression_data, tmpdir)
run_mcpcounter(expression_data, tmpdir)

Parsed with column specification:
cols(
  .default = col_double(),
  Gene_symbol = col_character()
)
See spec(...) for full column specifications.


,gerald.H5NKVBBXX.4.CGGCTATG.TATAGCCT,gerald.H5NKVBBXX.4.CTGAAGCT.ATAGAGGC,gerald.H5NKVBBXX.4.TAATGCGC.ATAGAGGC,gerald.H5NKVBBXX.4.TAATGCGC.TATAGCCT,gerald.H5NKVBBXX.4.TCTCGCGC.ATAGAGGC,gerald.H5NKVBBXX.5.ATTACTCG.ATAGAGGC,gerald.H5NKVBBXX.5.ATTACTCG.CCTATCCT,gerald.H5NKVBBXX.5.ATTACTCG.TATAGCCT,gerald.H5NKVBBXX.5.ATTCAGAA.ATAGAGGC,gerald.H5NKVBBXX.5.CGCTCATT.ATAGAGGC,⋯,gerald.H5NKVBBXX.6.TCCGCGAA.ATAGAGGC,gerald.H5NKVBBXX.6.TCCGCGAA.TATAGCCT,gerald.H5NKVBBXX.6.TCTCGCGC.TATAGCCT,gerald.H5NKVBBXX.7.ATTCAGAA.TATAGCCT,gerald.H5NKVBBXX.7.CGCTCATT.TATAGCCT,gerald.H5NKVBBXX.7.CTGAAGCT.TATAGCCT,gerald.H5NKVBBXX.7.GAATTCGT.TATAGCCT,gerald.H5NKVBBXX.7.GAGATTCC.ATAGAGGC,gerald.H5NKVBBXX.7.GAGATTCC.TATAGCCT,gerald.H5NKVBBXX.7.TCCGGAGA.CCTATCCT
A1BG,160.2753,29.19838,89.983320,172.5639,118.00680,365.7986,231.9847,458.5096,425.8787,1176.1113,⋯,578.7028,2938.4187,133.3406,449.5915,1032.19420,1121.0535,1016.4681,931.8012,966.2057,300.1420
A1CF,103.0000,9.00387,6.999996,208.0000,76.01059,288.0374,16.0156,261.2861,144.5596,134.9335,⋯,166.0010,183.0124,106.0640,125.0098,15.00308,130.5271,298.9732,222.0008,273.0109,450.0056
A2M,53163.7239,6305.79310,6032.217320,2788.6935,12548.85321,14220.8857,52604.2210,39731.8083,53494.5108,20731.3946,⋯,55476.2603,6677.8170,7270.9104,85258.1091,5215.15598,20041.5642,26814.7081,25570.4671,93369.1144,50277.6901


,gerald.H5NKVBBXX.4.CGGCTATG.TATAGCCT,gerald.H5NKVBBXX.4.CTGAAGCT.ATAGAGGC,gerald.H5NKVBBXX.4.TAATGCGC.ATAGAGGC,gerald.H5NKVBBXX.4.TAATGCGC.TATAGCCT,gerald.H5NKVBBXX.4.TCTCGCGC.ATAGAGGC,gerald.H5NKVBBXX.5.ATTACTCG.ATAGAGGC,gerald.H5NKVBBXX.5.ATTACTCG.CCTATCCT,gerald.H5NKVBBXX.5.ATTACTCG.TATAGCCT,gerald.H5NKVBBXX.5.ATTCAGAA.ATAGAGGC,gerald.H5NKVBBXX.5.CGCTCATT.ATAGAGGC,⋯,gerald.H5NKVBBXX.6.TCCGCGAA.ATAGAGGC,gerald.H5NKVBBXX.6.TCCGCGAA.TATAGCCT,gerald.H5NKVBBXX.6.TCTCGCGC.TATAGCCT,gerald.H5NKVBBXX.7.ATTCAGAA.TATAGCCT,gerald.H5NKVBBXX.7.CGCTCATT.TATAGCCT,gerald.H5NKVBBXX.7.CTGAAGCT.TATAGCCT,gerald.H5NKVBBXX.7.GAATTCGT.TATAGCCT,gerald.H5NKVBBXX.7.GAGATTCC.ATAGAGGC,gerald.H5NKVBBXX.7.GAGATTCC.TATAGCCT,gerald.H5NKVBBXX.7.TCCGGAGA.CCTATCCT
T cells,258.03017,42.369740,63.672543,158.37900,565.26341,383.62473,127.12544,512.77558,343.97235,1190.88239,⋯,241.35106,290.10382,252.31332,240.35471,293.85645,369.86630,868.87871,503.36622,423.55900,968.3559
CD8 T cells,128.41972,4.993360,20.999990,229.00000,152.95100,25.99999,0.00000,329.00000,160.25034,232.92266,⋯,163.30978,120.42475,178.99980,104.09030,340.22100,76.96831,395.32420,155.34520,154.45260,537.8959
Cytotoxic lymphocytes,41.58387,6.529693,20.730223,79.68378,37.48210,40.49905,17.29510,99.89621,69.86164,210.39135,⋯,66.89683,164.70929,12.41965,39.87661,81.40726,89.78837,131.80791,75.51874,74.95644,291.7428
NK cells,24.13744,5.088166,5.988858,25.92171,35.52820,11.17112,22.66280,73.33534,49.77071,56.60621,⋯,32.91442,28.05641,39.41052,55.03378,15.93965,99.74961,81.88579,59.23897,83.26667,116.5634
B lineage,257.07394,50.423209,51.173434,108.10623,120.98494,160.49692,65.32364,503.08778,308.02022,3976.57277,⋯,181.71993,300.57121,130.78001,204.85755,61.03991,790.23599,1065.91892,178.03942,510.20766,408.2572
Monocytic lineage,703.59358,72.582399,178.045856,281.70732,398.72884,577.25072,631.06060,1181.59918,1053.61526,1570.90034,⋯,1120.10784,692.87658,233.31712,1012.37858,573.92779,2873.39554,1851.19389,1001.83503,1750.94157,2951.6945
Myeloid dendritic cells,106.68688,11.300008,33.141178,86.77015,51.25082,33.80558,43.10429,230.01630,222.03981,199.72563,⋯,71.56706,79.20291,90.80000,128.79609,81.20010,161.10536,340.64179,136.37353,173.94844,410.6533
Neutrophils,659.49477,105.116185,101.384593,677.08543,1313.08040,1014.35809,506.61231,1136.58230,1142.73634,1454.25332,⋯,1136.78921,1131.13038,1130.55919,1140.68556,719.51374,3151.74898,1002.71661,1140.13750,1048.61663,2333.4675
Endothelial cells,1334.69317,159.916302,322.561283,340.47808,2242.32781,1609.33176,1947.32990,2005.12118,2133.78842,2058.15472,⋯,1559.55271,1550.30196,1034.04410,1806.33275,807.04314,1039.17906,1719.24469,1624.81512,2771.81321,2860.8877
Fibroblasts,17104.38284,1613.102243,23874.126032,4953.25464,8049.40722,16290.57799,113795.81921,121623.82084,51247.94948,57410.96679,⋯,70305.38662,48624.88255,7106.69414,47844.09478,46531.59456,92087.02762,16565.82265,98849.93688,131182.85049,26317.6356
